## Text Summarization

In [8]:
# Importing required libraries, APIs
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')
llm=ChatGroq(model="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E364D191C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E364CCAB40>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
from langchain.schema import (SystemMessage,AIMessage,HumanMessage)

speech="""
A warm welcome to all the notable, respected teachers, seniors, and friends. Today I am standing before you all to present a speech on AI, the rapidly evolving world of artificial intelligence, and its implications for our society.

Artificial Intelligence or AI is a simulation or a model of human intelligence in machines programmed that think like humans and mimic their actions. The word might bring to humans a robot from fiction stories of Science but in a real way, AI is already all around us, in our smartphones, digital assistants, feeds of social media, and many other aspects of daily life.

Friends, do you know that the origins of AI research go back to the beginning of 1950 when John Von Neumann and Alan Turing transformed computers with the use of values from 0 to 9, or decimal logic, and machines to binary logic? Furthermore, thanks to the advancements in computer processing power, the availability of big data, and increased funding in research, which helped bring breakthroughs in artificial intelligence. 
"""

In [10]:
chat_message=[SystemMessage("You are an assistant for summarizing speeches."),
              HumanMessage(f"Please provide short and concise summary of the speech:\n{speech}")
              ]

In [11]:
# Number of tokens in speech
llm.get_num_tokens(speech)

204

In [12]:
llm(chat_message)

AIMessage(content='This speech introduces artificial intelligence (AI), defining it as a simulation of human intelligence in machines. \n\nThe speaker highlights that AI is already present in everyday life, from smartphones to social media.  \n\nThe speech then briefly touches on the history of AI research, mentioning key figures like John Von Neumann and Alan Turing, and the factors that have contributed to recent breakthroughs in the field.  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 232, 'total_tokens': 313, 'completion_time': 0.147272727, 'prompt_time': 0.011960142, 'queue_time': 0.0034892359999999997, 'total_time': 0.159232869}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-71669808-d65f-46c3-acdd-ad3b7be1e22f-0', usage_metadata={'input_tokens': 232, 'output_tokens': 81, 'total_tokens': 313})

In [13]:
# Prompts for texts or speeches
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

generic_temp="""
Write Summary of the following speech in {language} language in concise way.
Speech: {speech}
"""
prompt=PromptTemplate(input_variables=["speech","language"],template=generic_temp)

In [14]:
complete_prompt=prompt.format(speech=speech,language="Hindi")
llm.get_num_tokens(complete_prompt)

223

In [15]:
llm_chain=LLMChain(llm=llm,prompt=prompt)
summary=llm_chain.run({"speech":speech,"language":"Hindi"})
summary

C:\Users\rkdat\AppData\Local\Temp\ipykernel_9108\3766749775.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain=LLMChain(llm=llm,prompt=prompt)
C:\Users\rkdat\AppData\Local\Temp\ipykernel_9108\3766749775.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary=llm_chain.run({"speech":speech,"language":"Hindi"})


'यह भाषण कृत्रिम बुद्धिमत्ता (AI) के बारे में है।  \n\nभाषणकार कहते हैं कि AI मशीनों में मानव बुद्धिमत्ता का अनुकरण है जो मानवों की तरह सोचती हैं और कार्य करती हैं। AI पहले से ही हमारे स्मार्टफोन, डिजिटल सहायकों और सोशल मीडिया फीड में मौजूद है। \n\n1950 के दशक में जॉन वॉन न्यूमैन और अलन ट्यूरिंग ने कंप्यूटरों के विकास में महत्वपूर्ण योगदान दिया जिससे AI में प्रगति हुई। \n\n\nबढ़ती कंप्यूटर शक्ति, बड़े डेटा की उपलब्धता और AI अनुसंधान में निवेश ने AI में क्रांति ला दी है। \n\n'

### Stuff Document Chain Text Summarization

In [16]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("leg.pdf")
docs=loader.load_and_split()
docs

[Document(metadata={'source': 'leg.pdf', 'page': 0}, page_content='LEGALNOTICE\nDate: 14November2024\nTo,Mr. AbhishekS.RDepartment of ComputerScience,SRMUniversity, KattankulathurCampus,Chennai, Tamil Nadu.\nSubject: CeaseandDesist NoticeforUnauthorizedCopyingandUsageof Original ProjectMaterial\nDearMr. AbhishekS.R,\nI, SreyasSai, astudent of SRMUniversity, Department of ComputerScience, amwritingthislegal noticetoaddressaseriousconcernregardingtheunauthorizedcopyingandusageofmyoriginal project titled"PitLaneInsiders,"whichwaspresentedaspart of myacademiccoursework.\nDetailsof Infringement: It hascometomyattentionthat youhavecopiedsubstantial portionsof myproject, includingbut not limitedtoitsconcept, analysis, andpresentationcontent, andpresentedit asyourownworkduringarecent collegepresentation. Thisact constitutesplagiarismandinfringement of intellectual property, violatingbothacademicguidelinesofSRMUniversityandapplicablecopyright laws.\nViolationof Rights:\n1. Intellectual Property

In [17]:
# Prompt template
template=""" 
Write a concise and short summary of following text.
text:{text}
"""

prompt=PromptTemplate(input_variables=['text'],template=template)

In [18]:
# Chains
from langchain.chains.summarize import load_summarize_chain
chain=load_summarize_chain(llm,chain_type="stuff",prompt=prompt,verbose=True)
output=chain.run(docs)
output



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
Write a concise and short summary of following text.
text:LEGALNOTICE
Date: 14November2024
To,Mr. AbhishekS.RDepartment of ComputerScience,SRMUniversity, KattankulathurCampus,Chennai, Tamil Nadu.
Subject: CeaseandDesist NoticeforUnauthorizedCopyingandUsageof Original ProjectMaterial
DearMr. AbhishekS.R,
I, SreyasSai, astudent of SRMUniversity, Department of ComputerScience, amwritingthislegal noticetoaddressaseriousconcernregardingtheunauthorizedcopyingandusageofmyoriginal project titled"PitLaneInsiders,"whichwaspresentedaspart of myacademiccoursework.
Detailsof Infringement: It hascometomyattentionthat youhavecopiedsubstantial portionsof myproject, includingbut not limitedtoitsconcept, analysis, andpresentationcontent, andpresentedit asyourownworkduringarecent collegepresentation. Thisact constitutesplagiarismandinfringement of intellectual property, violatingbothacademicguideli

'Sreyas Sai, a Computer Science student at SRM University, is issuing a cease and desist notice to Abhishek S.R. for plagiarizing Sai\'s original project, "PitLaneInsiders." Sai accuses Abhishek of copying substantial portions of the project, including its concept, analysis, and presentation content, for a recent college presentation. \n\nSai demands Abhishek cease all use of the project material, provide a written apology, notify relevant authorities, and retract any recognition received based on the copied work. Failure to comply within 7 days will result in legal action, including a formal complaint to the university administration and potential lawsuits for intellectual property infringement. \n\n\n'

### Map reduce to summarize large docs

In [19]:
# Text splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=100)
docs=text_splitter.split_documents(docs)

In [20]:
# prompt template
chunk_template=""" 
Summarize the below text.\n
Text:{text}
summary:
"""

map_prompt_template=PromptTemplate(input_variables=['text'],template=chunk_template)

In [21]:
final_prompt="""Provide final summary of entire text with these important points.
Add a Motivation Title, Start the precise summary with an introduction and provide the 
summary in number points for text. 
text: {text}
"""

final_prompt_template=PromptTemplate(input_variables=["text"],template=final_prompt)

In [22]:
summary_chain=load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt_template,
    combine_prompt=final_prompt_template,
    verbose=True
)

output=summary_chain.run(docs)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
Summarize the below text.

Text:LEGALNOTICE
Date: 14November2024
To,Mr. AbhishekS.RDepartment of ComputerScience,SRMUniversity, KattankulathurCampus,Chennai, Tamil Nadu.
Subject: CeaseandDesist NoticeforUnauthorizedCopyingandUsageof Original ProjectMaterial
DearMr. AbhishekS.R,
I, SreyasSai, astudent of SRMUniversity, Department of ComputerScience, amwritingthislegal noticetoaddressaseriousconcernregardingtheunauthorizedcopyingandusageofmyoriginal project titled"PitLaneInsiders,"whichwaspresentedaspart of myacademiccoursework.
Detailsof Infringement: It hascometomyattentionthat youhavecopiedsubstantial portionsof myproject, includingbut not limitedtoitsconcept, analysis, andpresentationcontent, andpresentedit asyourownworkduringarecent collegepresentation. Thisact constitutesplagiarismandinfringement of intellectual property, violatingbothacademicguidelinesofSRMUniversityanda

### Refine Chain for summarization

In [23]:
chain=load_summarize_chain(
    llm=llm,
    chain_type="refine",
    verbose=True,
)

output=chain.run(docs)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"LEGALNOTICE
Date: 14November2024
To,Mr. AbhishekS.RDepartment of ComputerScience,SRMUniversity, KattankulathurCampus,Chennai, Tamil Nadu.
Subject: CeaseandDesist NoticeforUnauthorizedCopyingandUsageof Original ProjectMaterial
DearMr. AbhishekS.R,
I, SreyasSai, astudent of SRMUniversity, Department of ComputerScience, amwritingthislegal noticetoaddressaseriousconcernregardingtheunauthorizedcopyingandusageofmyoriginal project titled"PitLaneInsiders,"whichwaspresentedaspart of myacademiccoursework.
Detailsof Infringement: It hascometomyattentionthat youhavecopiedsubstantial portionsof myproject, includingbut not limitedtoitsconcept, analysis, andpresentationcontent, andpresentedit asyourownworkduringarecent collegepresentation. Thisact constitutesplagiarismandinfringement of intellectual property, violatingbothacademicguidelinesofSRMUniver